In [1]:
import heapq
import copy
import time

depth_0 = [[1, 2, 3],
           [4, 5, 6],
           [7, 8, 0]]

depth_2 = [[1, 2, 3],
           [4, 5, 6],
           [0, 7, 8]]

depth_4 = [[1, 2, 3],
           [5, 0, 6],
           [4, 7, 8]]

depth_8 = [[1, 3, 6],
           [5, 0, 2],
           [4, 7, 8]]

depth_12 = [[1, 3, 6],
           [5, 0, 7],
           [4, 8, 2]]

depth_16 = [[1, 6, 7],
           [5, 0, 3],
           [4, 8, 2]]

depth_20 = [[7, 1, 2],
           [4, 8, 5],
           [6, 3, 0]]

depth_24 = [[0, 7, 2],
           [4, 6, 1],
           [3, 5, 8]]

test_cases = {'depth_0': depth_0,
              'depth_2': depth_2,
              'depth_4': depth_4,
              'depth_8': depth_8,
              'depth_12': depth_12,
              'depth_16': depth_16,
              'depth_20': depth_20,
              'depth_24': depth_24
             }
#test_cases

In [2]:
# random_1 = [[0, 1, 3],
#             [4, 2, 6],
#             [7, 5, 8]]
# random_2 = [[2, 3, 0],
#             [1, 5, 6],
#             [4, 7, 8]]
# random_3 = [[2, 3, 6],
#             [1, 0, 5],
#             [4, 7, 8]]
# random_4 = [[2, 3, 6],
#             [4, 1, 5],
#             [7, 0, 8]]

# random_cases = {'random_1': random_1,
#               'random_2': random_2,
#               'random_3': random_3,
#               'random_4': random_4
#              }
# random_cases

In [3]:
def select_heuristic():
    print("Select Heuristic Algorithm to solve the puzzle")
    print("\t 1 for Uniform Cost")
    print("\t 2 for Misplaced Tiles Heuristic") 
    print("\t 3 for Manhattan Distance Heuristic")
    
    heuristic_selection = int(input())
    if heuristic_selection == 1:
        heuristic = 'Uniform cost'
    elif heuristic_selection == 2:
        heuristic = 'Misplaced tiles heuristic'
    elif heuristic_selection == 3:
        heuristic = 'Manhattan distance heuristic'
    else:
        heuristic = '4'
        
    return heuristic

def select_test_cases_from_default():
    print("Select test cases from default cases")
    print("\t 1 for 'trivial' test case")
    print("\t 2 for 'veryEasy' test case") 
    print("\t 3 for 'easy' test case")
    print("\t 4 for 'doable' test case")
    print("\t 5 for 'oh_boy!' test case")
    
    test_selection = int(input())
    
        
    return test_selection

def input_test_case():
    print("Please input three rows one by one. Use space between each values. Consider 0 as blank cell")
    row1 = input("Row1: ").split(" ")
    row1 = [int(val) for val in row1]
    
    row2 = input("Row2: ").split(" ")
    row2 = [int(val) for val in row2]
    
    row3 = input("Row3: ").split(" ")
    row3 = [int(val) for val in row3]
    
    test_case = [row1, row2, row3]
    
    return test_case

In [4]:
class PuzzleState:
    def __init__(self, board):
        self.board = board
        self.blank_row, self.blank_col = self.get_blank_cell()
        
    def get_blank_cell(self):
        for i in range(3):
            for j in range(3):
                if self.board[i][j] == 0:
                    return i, j

    def is_goal_state(self):
        for i in range(3):
            for j in range(3):
                if i == 2 and j == 2:
                    if self.board[i][j] != 0:
                        return False
                elif self.board[i][j] != (i * 3) + j + 1:
                    return False
        return True

    def __lt__(self, other):
        for i in range(3):
            for j in range(3):
                if self.board[i][j] < other.board[i][j]:
                    return True
                if self.board[i][j] > other.board[i][j]:
                    return False
        return False

    def get_neighbors(self):
        # Get the blank tile's row and column.
        blank_row = self.blank_row
        blank_col = self.blank_col

        # Get the list of possible moves.
        possible_moves = [(-1, 0), (0, -1), (1, 0), (0, 1)]
        
        # Create a list of the neighbors.
        neighbors = []
        for move in possible_moves:
            new_row = blank_row + move[0]
            new_col = blank_col + move[1]

            # Check if the move is valid.
            if new_row >= 0 and new_row < 3 and new_col >= 0 and new_col < 3:
                # Copy the current board and make the move.
                new_board = copy.deepcopy(self.board)
                new_board[blank_row][blank_col] = self.board[new_row][new_col]
                new_board[new_row][new_col] = self.board[blank_row][blank_col]
                
                neighbors.append(PuzzleState(new_board))

        return neighbors

    def misplaced_tiles_heuristic(self):
        count = 0
        for i in range(3):
            for j in range(3):
                target_cell_value = (i * 3) + j + 1
                if self.board[i][j] != target_cell_value and self.board[i][j] != 0:
                    count += 1
        return count

    def manhattan_distance_heuristic(self):
        distance = 0
        for i in range(3):
            for j in range(3):
                cell_value = self.board[i][j]
                if cell_value != 0:
                    row = (cell_value - 1) // 3
                    col = (cell_value - 1) % 3
                    
                    # distance = row distance + col distance
                    distance += abs(row - i) + abs(col - j)
        return distance
    
    def get_string(self):
        dummy_string = ""
        for i in range(3):
            for j in range(3):
                dummy_string += str(self.board[i][j])
        return dummy_string


In [5]:
def printBoard(board):
    for row in board:
        print(row)

In [6]:
def make_queue(start_state):
    state_queue = []
    heapq.heappush(state_queue, (0, start_state, 0))
    
    return state_queue

def pop_queue(state_queue):
    f_n, state, depth = heapq.heappop(state_queue)
    
    return f_n, state, depth

def insert_to_queue(state_queue, f_n, neighbor, depth):
    heapq.heappush(state_queue, (f_n, neighbor, depth))

In [7]:
def solve_eight_puzzle(start_state, heuristic, verbose):
    start_time = time.time()
    # Create a priority queue to store the open states.
    state_queue = make_queue(start_state)
    
    # Create a set to store the visited_states.
    visited_states = set()
    
    # Initialize max queue size, depth, and number of expanded nodes
    max_queue_size = 1
    max_depth = 0
    num_expanded_nodes = 0
    
    solved = False
    # While the queue is not empty:
    while state_queue:
        # Get the state with the lowest f(n) value from the  queue.
        f_n, state, depth = pop_queue(state_queue) 
        
        # If the state is the goal state, return it.
        if state.is_goal_state():
            solved = True
            print("Puzzle Solved: ")
            printBoard(state.board)
            break

        # if state is not goal state, expand it
        # Calculate g(n) and h(n) for the state
        g_n = depth
        if heuristic == 'Uniform cost':
            h_n = 0
        elif heuristic == 'Misplaced tiles heuristic':
            h_n = state.misplaced_tiles_heuristic()
        elif heuristic == 'Manhattan distance heuristic':
            h_n = state.manhattan_distance_heuristic()

        # Print the state to be expanded
        if verbose == 2:
            print("The best state to expand with a g(n) =", g_n, "and h(n) =", h_n, "is")
            printBoard(state.board)
        
        # Add the state to the visited_states set.
        visited_states.add(state.get_string())
        num_expanded_nodes += 1

        
        for neighbor in state.get_neighbors():
            # If the neighbor is not in the visited_states set:
            if neighbor.get_string() not in visited_states:
                # Calculate the g(n) and h(n) values for the neighbor.
                g_n = depth + 1
                if heuristic == 'Uniform cost':
                    h_n = 0
                elif heuristic == 'Misplaced tiles heuristic':
                    h_n = neighbor.misplaced_tiles_heuristic()
                elif heuristic == 'Manhattan distance heuristic':
                    h_n = neighbor.manhattan_distance_heuristic()

                # Calculate the f(n) value for the neighbor.
                f_n = g_n + h_n

                # Add the neighbor to the open queue.
                insert_to_queue(state_queue, f_n, neighbor, depth + 1)
                
                # Update max queue size if necessary
                max_queue_size = max(max_queue_size, len(state_queue))

                # Update max depth if necessary
                max_depth = max(max_depth, depth + 1)
                
    
    end_time = time.time()
    total_time = end_time - start_time
    return solved, max_queue_size, depth, num_expanded_nodes, total_time

In [ ]:
if __name__ == "__main__":
    while(True):
        print("Welcome to 8-puzzle game")
        heuristic = select_heuristic()
        print(heuristic)
        if heuristic == '4':
            print("Sorry, you have mistyped to select heuristic")
            print("Please select either 1 or 2 or 3 to select heuristic algorith")
            continue
        
        print("Please select or input test cases")
        print("\t 1 to select default test cases")
        print("\t 2 to create your own puzzle")
        
        default_or_create = int(input())
        
        if default_or_create == 1:
            print("Want to see detailed steps?")
            print("\t Type 1 to see minimum \n \t Type 2 to see maximum")
            verbose = int(input())
            
            for name, test_board in test_cases.items():
                print("Test case: ", name)
                printBoard(test_board)
                
                state = PuzzleState(test_board)

                is_solved, max_queue_size, max_depth, num_expanded_nodes, total_time\
                = solve_eight_puzzle(state, heuristic, verbose)
                print("Solved: ", is_solved)
                #printBoard(solved_board)
                print("max queue: ", max_queue_size, " depth: ", max_depth, 
                      " num expanded nodes: ", num_expanded_nodes, " total time need: ", total_time)
                print("\n\n")
        elif default_or_create == 2:
            print("Please create your own test case:")
            test_board = input_test_case()
            
            print("Want to see detailed steps?")
            print("\t Type 1 to see minimum \n \t Type 2 to see maximum")
            verbose = int(input())
            
            printBoard(test_board)
            
            state = PuzzleState(test_board)
            
            is_solved, max_queue_size, max_depth, num_expanded_nodes, total_time\
            = solve_eight_puzzle(state, heuristic, verbose)
            print("Solved: ", is_solved)
            #printBoard(solved_board)
            print("max queue: ", max_queue_size, " depth: ", max_depth, 
                  " num expanded nodes: ", num_expanded_nodes, " total time need: ", total_time)
            
        else:
            print("Wrong choice. Input either 1 or 2")
            
        print("\n\n")

Welcome to 8-puzzle game
Select Heuristic Algorithm to solve the puzzle
	 1 for Uniform Cost
	 2 for Misplaced Tiles Heuristic
	 3 for Manhattan Distance Heuristic


 2


Misplaced tiles heuristic
Please select or input test cases
	 1 to select default test cases
	 2 to create your own puzzle


 2


Please create your own test case:
Please input three rows one by one. Use space between each values. Consider 0 as blank cell


Row1:  1 2 3
Row2:  4 5 6
Row3:  0 7 8


Want to see detailed steps?
	 Type 1 to see minimum 
 	 Type 2 to see maximum


 1


[1, 2, 3]
[4, 5, 6]
[0, 7, 8]
Puzzle Solved: 
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]
Solved:  True
max queue:  3  depth:  2  num expanded nodes:  2  total time need:  0.00042510032653808594



Welcome to 8-puzzle game
Select Heuristic Algorithm to solve the puzzle
	 1 for Uniform Cost
	 2 for Misplaced Tiles Heuristic
	 3 for Manhattan Distance Heuristic
